<a href="https://colab.research.google.com/github/JSJeong-me/AI-Innovation-2024/blob/main/NLP/4-4-CromaDB-Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers chromadb pandas

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.utils import embedding_functions

In [4]:
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1nUjOtPsWy17U-m-HjkMBpDdz-l9t2c2C' -O ./fine_food_reviews_1k.csv

--2024-10-07 08:46:45--  https://drive.google.com/uc?export=download&id=1nUjOtPsWy17U-m-HjkMBpDdz-l9t2c2C
Resolving drive.google.com (drive.google.com)... 142.251.2.101, 142.251.2.138, 142.251.2.139, ...
Connecting to drive.google.com (drive.google.com)|142.251.2.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1nUjOtPsWy17U-m-HjkMBpDdz-l9t2c2C&export=download [following]
--2024-10-07 08:46:45--  https://drive.usercontent.google.com/download?id=1nUjOtPsWy17U-m-HjkMBpDdz-l9t2c2C&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.101.132, 2607:f8b0:4023:c06::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.250.101.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 436148 (426K) [application/octet-stream]
Saving to: ‘./fine_food_reviews_1k.csv’

./fine_food_reviews 100%[===================>] 425.93K 

In [5]:
# Load the data
import pandas as pd
df = pd.read_csv('./fine_food_reviews_1k.csv')

In [6]:


# Initialize the embedding model (using a smaller model for demonstration purposes)
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

In [7]:


# Generate embeddings for the 'Text' column
embeddings = embedding_model.encode(df['Text'].tolist())

# Initialize ChromaDB client and create a collection
client = chromadb.Client()
collection = client.create_collection(name="reviews")

# Insert data into the ChromaDB collection
collection.add(
    embeddings=embeddings,
    metadatas=[{'text': text, 'summary': summary} for text, summary in zip(df['Text'], df['Summary'])],
    ids=[str(i) for i in df.index]
)

# Sample search using a query
query_text = "good quality product"
query_embedding = embedding_model.encode([query_text])[0]

# Perform the search
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=5,
    include=['metadatas', 'distances']
)

# Display the search results
print(results)


{'ids': [['816', '563', '384', '900', '149']], 'distances': [[0.9377105236053467, 0.9595870971679688, 1.0355348587036133, 1.0388377904891968, 1.050167441368103]], 'metadatas': [[{'summary': 'Great product!!!!!!!!!!!!', 'text': 'price and quality are awesome!! Buy recommendation only!!!!!!!!!!!!!!!! We ordered several of them and we loved them.  Price is so great and they offer free shipping if you buy certain amount.'}, {'summary': 'Good stuff', 'text': "Very good product, helps with a lot of ailments. It's great help stocking up on vitimain that you are deficient in."}, {'summary': 'Great taste', 'text': 'This product is great.  My husband and I both love it.  It is good as a supplement, and for cooking.  It has a wonderful scent and a soft taste.  It is a very healthy product.  We are pleased.'}, {'summary': 'Excellent', 'text': "Great product at a good price. Will definitely be ordering this product again once it's all used up and the bottles empty."}, {'summary': 'buttons', 'text':

In [8]:
# Sample search using a query
query_text = "pet food"
query_embedding = embedding_model.encode([query_text])[0]

# Perform the search
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=5,
    include=['metadatas', 'distances']
)

# Display the search results
print(results)

{'ids': [['818', '444', '127', '752', '184']], 'distances': [[0.7441419959068298, 0.8426192998886108, 0.89345782995224, 0.9527755975723267, 0.9711098074913025]], 'metadatas': [[{'summary': 'Good food', 'text': 'The only dry food my queen cat will eat. Helps prevent hair balls. Good packaging. Arrives promptly. Recommended by a friend who sells pet food.'}, {'summary': 'Healthy Dog Food', 'text': 'This is a very healthy dog food. Good for their digestion. Also good for small puppies. My dog eats her required amount at every feeding.'}, {'summary': 'Great for small breeds with sensitive systems', 'text': 'I have two small mixed breed dogs each under 10 pounds. This is to only food I have found that they can eat with out any problems.'}, {'summary': 'Tepid Response from My Cat', 'text': "It's difficult to write a review for food that I have not personally eaten or been able to obtain a verbal response for from its intended consumer. I don't think my normally voracious cat loved this produ

In [ ]:
# Initialize ChromaDB client
client = chromadb.Client()

# Try to get the collection, create it if it doesn't exist.
collection = client.get_or_create_collection(name="reviews")

# Insert data into the ChromaDB collection
collection.add(
    embeddings=embeddings,
    metadatas=[{'text': text, 'summary': summary} for text, summary in zip(df['Text'], df['Summary'])],
    ids=[str(i) for i in df.index]
)


In [10]:

# Function to convert distance to a descriptive word
def distance_to_word(distance):
    if distance < 0.2:
        return "very close"
    elif distance < 0.4:
        return "close"
    elif distance < 0.6:
        return "somewhat close"
    else:
        return "distant"


In [11]:
# Sample search using a query
query_text = "pet food"
query_embedding = embedding_model.encode([query_text])[0]

# Perform the search
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=5,
    include=['metadatas', 'distances'] # removed 'documents' from include
)

# Access the 'documents' list directly from results
formatted_results = [
    {
        'text': res['text'], # Access text directly from the dictionary
        'summary': res['summary'], # Access summary directly from the dictionary
        'distance': distance_to_word(dist) # Use dist from enumerate
    }
    for res, dist in zip(results['metadatas'][0], results['distances'][0]) # Iterate through metadatas and distances using zip
]

In [12]:

# Display the formatted results
for result in formatted_results:
    print(f"Summary: {result['summary']}, Distance: {result['distance']}")
    print(f"Text: {result['text']}\n")

Summary: Good food, Distance: distant
Text: The only dry food my queen cat will eat. Helps prevent hair balls. Good packaging. Arrives promptly. Recommended by a friend who sells pet food.

Summary: Healthy Dog Food, Distance: distant
Text: This is a very healthy dog food. Good for their digestion. Also good for small puppies. My dog eats her required amount at every feeding.

Summary: Great for small breeds with sensitive systems, Distance: distant
Text: I have two small mixed breed dogs each under 10 pounds. This is to only food I have found that they can eat with out any problems.

Summary: Tepid Response from My Cat, Distance: distant
Text: It's difficult to write a review for food that I have not personally eaten or been able to obtain a verbal response for from its intended consumer. I don't think my normally voracious cat loved this product though; she would often take a few bites, then walk away from it, then come back to it only when she became very hungry. Of course, this is 